In [24]:
# Dependencies and Setup
import pandas as pd
import numpy as np

# File to Load (Remember to Change These)
file_to_load = "Resources/purchase_data.csv"

# Read Purchasing File and store into Pandas data frame
purchase_data = pd.read_csv(file_to_load)

In [25]:
purchase_data = purchase_data.dropna(how='any')
people = len(purchase_data['SN'].unique())
people

576

In [26]:
purchase_data.head()

,Purchase ID,SN,Age,Gender,Item ID,Item Name,Price
0,0,Lisim78,20,Male,108,"Extraction, Quickblade Of Trembling Hands",3.53
1,1,Lisovynya38,40,Male,143,Frenzied Scimitar,1.56
2,2,Ithergue48,24,Male,92,Final Critic,4.88
3,3,Chamassasya86,24,Male,100,Blindscythe,3.27
4,4,Iskosia90,23,Male,131,Fury,1.44


In [27]:
num_items = len(purchase_data['Item ID'].unique())
avg_price = purchase_data['Price'].mean()
num_purchases= len(purchase_data)
revenue = purchase_data['Price'].sum()
Purchasing_Analysis = [{'Number of Unique Items': num_items,
                        'Average Price': avg_price,
                        'Number of Purchases': num_purchases,
                        'Total Revenue': revenue}]
PA= pd.DataFrame(Purchasing_Analysis, columns=[
    'Number of Unique Items','Average Price',
    'Number of Purchases','Total Revenue'])
PA["Average Price"] = PA["Average Price"].map("${:.2f}".format)  
PA["Total Revenue"] = PA["Total Revenue"].map("${:.2f}".format)  
PA


,Number of Unique Items,Average Price,Number of Purchases,Total Revenue
0,183,$3.05,780,$2379.77


In [28]:
#purchase_data['Gender'].value_counts()
Purchasers = purchase_data.drop_duplicates(subset=['SN'], keep='first')
gender = pd.DataFrame(Purchasers['Gender'].value_counts())
gender = gender.rename(columns= {'Gender':'Total Count'})
gender['Percentage of Players'] = (gender['Total Count']/len(Purchasers))*100
gender['Percentage of Players'] = gender['Percentage of Players'].map("{:.2f}%".format)  
gender


,Total Count,Percentage of Players
Male,484,84.03%
Female,81,14.06%
Other / Non-Disclosed,11,1.91%


In [29]:
#Use the original purchase_data dataframe and groupby Gender 
Gender_Purchases = purchase_data.groupby(['Gender'])
gender_purchase_count = Gender_Purchases['Gender'].count()
gender_avg_purchase_price = Gender_Purchases['Price'].mean()
gender_total_purchases = Gender_Purchases['Price'].sum()

#For per person statistics you much us the umique Purchasers data frame
avg_per_person = Gender_Purchases['Price'].sum()/Purchasers['Gender'].value_counts()

#Built DataFrame of all the computed gender specific statistics
PA_Gender = pd.DataFrame({'Purchase Count': gender_purchase_count,
                          'Average Purchase Price':gender_avg_purchase_price,
                         'Total Purchase Value':gender_total_purchases,
                         'Average Total Purchase per Person': avg_per_person})

#Format Results
PA_Gender['Average Purchase Price'] = PA_Gender['Average Purchase Price'].map("${:.2f}".format)
PA_Gender['Total Purchase Value'] = PA_Gender['Total Purchase Value'].map("${:.2f}".format)
PA_Gender['Average Total Purchase per Person'] = PA_Gender['Average Total Purchase per Person'].map("${:.2f}".format)

PA_Gender

,Purchase Count,Average Purchase Price,Total Purchase Value,Average Total Purchase per Person
Gender,,,,
Female,113,$3.20,$361.94,$4.47
Male,652,$3.02,$1967.64,$4.07
Other / Non-Disclosed,15,$3.35,$50.19,$4.56


In [30]:
#Use Purchasers df
# Create the bins for age in which Data will be held
#Bins are 0, 10, 15, 20, 25, 30, 35, max.   
bins = [0, 10, 15, 20, 25, 30, 35,40,max(Purchasers['Age'])+10]

# Create the names for the four bins
group_names = ["<10", "10-14", "15-19", "20-24", "25-29", "30-34", "35-39","40+"]
Purchasers['Age Group']= pd.cut(Purchasers["Age"], bins, labels=group_names, right=False)

# Creating a group based off of the bins
age_group = Purchasers.groupby('Age Group')

GroupByAge = age_group.count()
GroupByAge.rename(columns = {'Purchase ID':'Total Count'}, inplace= True)

#Calculate Percentage of Players and format it.
SumOfAges = GroupByAge['Total Count'].sum()
GroupByAge['Percentage of Players'] = ((GroupByAge['Total Count']/SumOfAges)*100).map("{:.2f}%".format)
GroupByAge = GroupByAge[['Total Count', 'Percentage of Players']]

GroupByAge

C:\Users\carri\Anaconda\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,Total Count,Percentage of Players
Age Group,,
<10,17,2.95%
10-14,22,3.82%
15-19,107,18.58%
20-24,258,44.79%
25-29,77,13.37%
30-34,52,9.03%
35-39,31,5.38%
40+,12,2.08%


In [31]:
#Group Names and Bins were defined in previous box
age_purchase_group= pd.cut(purchase_data["Age"], bins, labels=group_names, right=False)
purchase_data['Age Group'] = age_purchase_group

# Creating a group based off of the bins
age_purchase_group = purchase_data.groupby("Age Group")
GroupByAgePurchase = age_purchase_group.count()

GroupByAgePurchase.rename(columns = {'Purchase ID':'Purchase Count'}, inplace= True)
GroupByAgePurchase['Average Purchase Price']=(age_purchase_group['Price'].mean()).map("${:.2f}".format)
GroupByAgePurchase['Total Purchase Value']=(age_purchase_group['Price'].sum()).map("${:.2f}".format)

#Since there are duplicate purchaces divide the sum by unique number of Purchasers per age goup 
#which is defined in previous box.
Percent= age_purchase_group['Price'].sum()/GroupByAge['Total Count']

#Create new column for each item and format them
GroupByAgePurchase['Average Total Purchase per Person']=Percent.map("${:.2f}".format)
GroupByAgePurchase = GroupByAgePurchase[['Purchase Count',
                                         'Average Purchase Price',
                                         'Total Purchase Value',
                                         'Average Total Purchase per Person']]
GroupByAgePurchase.rename(columns = {'Purchase ID':'Purchase Count'}, inplace= True)

GroupByAgePurchase

,Purchase Count,Average Purchase Price,Total Purchase Value,Average Total Purchase per Person
Age Group,,,,
<10,23,$3.35,$77.13,$4.54
10-14,28,$2.96,$82.78,$3.76
15-19,136,$3.04,$412.89,$3.86
20-24,365,$3.05,$1114.06,$4.32
25-29,101,$2.90,$293.00,$3.81
30-34,73,$2.93,$214.00,$4.12
35-39,41,$3.60,$147.67,$4.76
40+,13,$2.94,$38.24,$3.19


In [32]:
#First derive count by user
purchase_count = purchase_data['SN'].value_counts()

#Take purchase_data and groupby user(SN).
group_spenders = purchase_data.groupby(['SN'])
avg_purchase_price = group_spenders['Price'].mean()
total_purchases = group_spenders["Price"].sum()

#Create dataframe with the statistics from above
Top_Spenders = pd.DataFrame({'Purchase Count':purchase_count,
                             'Average Purchase Price':avg_purchase_price,
                            'Total Purchase Value':total_purchases})

#Sort Data (Must sort before formating)
Top_Spenders = Top_Spenders.sort_values(['Total Purchase Value','Purchase Count'], ascending= False)

#Format Results 
Top_Spenders['Average Purchase Price'] = Top_Spenders['Average Purchase Price'].map("${:.2f}".format)
Top_Spenders['Total Purchase Value'] = Top_Spenders['Total Purchase Value'].map("${:.2f}".format)

Top_Spenders.head()

,Purchase Count,Average Purchase Price,Total Purchase Value
Lisosia93,5,$3.79,$18.96
Idastidru52,4,$3.86,$15.45
Chamjask73,3,$4.61,$13.83
Iral74,4,$3.40,$13.62
Iskadarya95,3,$4.37,$13.10


In [33]:
#Group by item
popular = purchase_data.groupby(['Item ID','Item Name'])

popular.head()

#Calculate statistical values
item_count = popular.count()
item_count = pd.DataFrame(item_count)


item_price = popular['Price'].mean()
purchase_value = popular['Price'].sum()
purchase_count = popular['Purchase ID'].count()

#Populate DataFrame with statistical values
Popular_Items = pd.DataFrame({'Purchase Count': purchase_count, 
                              'Item Price':item_price,
                              'Total Purchase Value':purchase_value})

Popular_Items.head()

,,Purchase Count,Item Price,Total Purchase Value
Item ID,Item Name,,,
0,Splinter,4,1.28,5.12
1,Crucifer,3,3.26,9.78
2,Verdict,6,2.48,14.88
3,Phantomlight,6,2.49,14.94
4,Bloodlord's Fetish,5,1.70,8.50


In [34]:
#All of this first section was done above for Popular Items

#Sort DataFrame on total purchase value decending
Profitable_Items = Popular_Items.sort_values('Total Purchase Value', ascending= False)

#Format DataFrame
Profitable_Items['Item Price'] = Profitable_Items['Item Price'].map('${:.2f}'.format)
Profitable_Items['Total Purchase Value'] = Profitable_Items['Total Purchase Value'].map('${:.2f}'.format)

Profitable_Items.head()


,,Purchase Count,Item Price,Total Purchase Value
Item ID,Item Name,,,
178,"Oathbreaker, Last Hope of the Breaking Storm",12,$4.23,$50.76
82,Nirvana,9,$4.90,$44.10
145,Fiery Glass Crusader,9,$4.58,$41.22
92,Final Critic,8,$4.88,$39.04
103,Singed Scalpel,8,$4.35,$34.80
